# Informal tests of sqlalchemy wrapper

In [1]:
%load_ext jupyter_anywidget_pglite
from jupyter_anywidget_pglite import pglite_headless

pg = pglite_headless()
pg.ready()

postgresWidget(headless=True, response={'status': 'initialising'})

In [2]:
%%pglite_magic -w pg -r
CREATE TABLE IF NOT EXISTS users (
            id SERIAL PRIMARY KEY ,
            name TEXT NOT NULL
)


{'status': 'completed',
 'response': {'rows': [], 'fields': [], 'affectedRows': 0},
 'response_type': 'single'}

In [3]:
%%pglite_magic -w pg -r
        CREATE TABLE IF NOT EXISTS test (
            id SERIAL PRIMARY KEY,
            user_id INTEGER NOT NULL,
            name TEXT NOT NULL
        )

{'status': 'completed',
 'response': {'rows': [], 'fields': [], 'affectedRows': 0},
 'response_type': 'single'}

In [4]:
%%pglite_magic -w pg -r
        CREATE TABLE IF NOT EXISTS orders (
            id SERIAL PRIMARY KEY,
            user_id INTEGER NOT NULL,
            product TEXT NOT NULL,
            FOREIGN KEY (user_id) REFERENCES users(id) ON DELETE CASCADE
        )

{'status': 'completed',
 'response': {'rows': [], 'fields': [], 'affectedRows': 0},
 'response_type': 'single'}

In [5]:
%%pglite_magic -w pg -r
INSERT INTO users (id,name) VALUES (2, 'Alice');

{'status': 'completed',
 'response': {'rows': [], 'fields': [], 'affectedRows': 1},
 'response_type': 'single'}

In [6]:
from jupyter_anywidget_pglite.sqlalchemy_api import create_engine
import pandas as pd
import logging

LOGGING=False

if LOGGING:
    logging.basicConfig(level=logging.DEBUG)
    handler = logging.StreamHandler()  # Logs to console
    formatter = logging.Formatter("%(levelname)s - %(message)s")
    handler.setFormatter(formatter)

    logger = logging.getLogger("jupyter_anywidget_pglite")#.setLevel(logging.DEBEG)
    logger.addHandler(handler)
    logger.propagate = False

    logging.getLogger("sqlalchemy.engine").setLevel(logging.DEBUG)

engine2 = create_engine(pg)
pd.read_sql("SELECT * FROM users;", engine2)

,id,name
0,2,Alice


In [7]:
from sqlalchemy import inspect

inspector = inspect(engine2)

In [8]:
inspector.get_table_names()

['users', 'test', 'orders']

In [9]:
inspector.has_table("users"), inspector.has_table("users2"),

(True, False)

In [10]:
o = inspector.get_columns("users")
assert o[0]["name"] == "id"
o

[{'name': 'id',
  'type': Integer(),
  'nullable': False,
  'default': "nextval('users_id_seq'::regclass)"},
 {'name': 'name', 'type': String(), 'nullable': False, 'default': None}]

In [11]:
o = inspector.get_pk_constraint("users")
assert o == {"constrained_columns": ["id"], "name": "pk_users"}
o

{'constrained_columns': ['id'], 'name': 'pk_users'}

In [12]:
o = inspector.get_foreign_keys("orders")
assert o == [
    {
        "name": "fk_orders_id",
        "constrained_columns": ["id"],
        "referred_schema": "public",
        "referred_table": "users",
        "referred_columns": ["orders_user_id_fkey"],
    }
]
o

[{'name': 'fk_orders_id',
  'constrained_columns': ['id'],
  'referred_schema': 'public',
  'referred_table': 'users',
  'referred_columns': ['orders_user_id_fkey']}]

In [13]:
inspector.get_indexes("users")

[{'name': 'users_pkey', 'column_names': ['id'], 'unique': True}]

In [14]:
o = inspector.get_unique_constraints("orders")
assert o == []
o

[]

In [15]:
o = inspector.get_check_constraints("orders")
assert o == []
o

[]

In [16]:
import pandas as pd
o = pd.read_sql("SELECT * FROM users;", engine2)
pd.testing.assert_frame_equal(o, pd.DataFrame({"id": [2], "name":[ "Alice"]}))
o

,id,name
0,2,Alice


In [17]:
o = pd.read_sql("SELECT version();",engine2)
assert "version" in o.columns
o

,version
0,"PostgreSQL 16.4 on x86_64-pc-linux-gnu, compil..."


In [18]:
# This should error; the table already exist, and by default we have if_exists="fail"
pd.DataFrame({"user_id": [8, 9, 10], "name": ["XBob", "XClaire", "XDave"]}).to_sql(
    name="test",  con=engine2, index=False
)

ValueError: Table 'test' already exists.

In [ ]:
# This should create a new table and populate it with three rows
o_df0 = pd.DataFrame({"user_id": [8, 9, 10], "name": ["XBob", "XClaire", "XDave"]})
o_df0.to_sql(
    name="test2", con=engine2, index=False
)

In [ ]:
o = pd.read_sql("SELECT * FROM test2;", engine2)
pd.testing.assert_frame_equal(o, o_df0)
o

In [19]:
# This should append three items to a pre-existing table
# index=False is not recognised?
o_df = pd.DataFrame({"user_id": [8, 9, 10], "name": ["Bob", "Claire", "Dave"]})
o_df.to_sql(
    name="test", con=engine2, if_exists="append", index=False
)

0

In [20]:
# This should display a table with three entries
o = pd.read_sql("SELECT * FROM test;", engine2)
assert len(o) ==len(o_df)
assert ["id"] + list(o_df.columns) == list(o.columns)
o

,id,user_id,name
0,1,8,Bob
1,2,9,Claire
2,3,10,Dave


In [21]:
# This should drop a pre-existing table, and replace it with one with new entries
o_df2 = pd.DataFrame({"user_id": [8, 9], "name": ["Bob2", "Claire2"]})
o_df2.to_sql(name="test", con=engine2, if_exists="replace", index=False)
# ‘fail’, ‘replace’, ‘append’
# Fails on replace

0

In [22]:
o = pd.read_sql("SELECT * FROM test;", engine2)
pd.testing.assert_frame_equal(o, o_df2)
o

,user_id,name
0,8,Bob2
1,9,Claire2


In [23]:
# Explicitly drop the table if it exists
with engine2.connect() as conn:
    conn.execute("DROP TABLE IF EXISTS test")
inspector.has_table("test")

False

In [24]:
engine2.dialect.server_version_info

(16, 4)

In [25]:
o_df3 = pd.DataFrame({"user_id": [8, 9], "name": ["Bob3", "Claire3"]})
o_df3.to_sql(name="test5", con=engine2, if_exists="replace", index=True)

0

In [26]:
o = pd.read_sql("SELECT * FROM test5;", engine2)
assert len(o) == len(o_df3)
assert ["index"] + list(o_df3.columns) == list(o.columns)
o

,index,user_id,name
0,0,8,Bob3
1,1,9,Claire3
